# Dataset Loader

The first part of the Open Bandit Pipeline (OBP) is the dataset loader. For the Open Bandit Dataset (OBD), the loader is `opb.dataset.OpenBanditDataset` ([docs](https://zr-obp.readthedocs.io/en/latest/_autosummary/obp.dataset.real.html#obp.dataset.real.OpenBanditDataset)). 

As with many classes in the OBP, the dataset modules are implemented with [dataclasses](https://docs.python.org/3.7/library/dataclasses.html).

The dataset module inherits from `obp.dataset.base.BaseRealBanditDatset` ([docs](https://zr-obp.readthedocs.io/en/latest/_autosummary/obp.dataset.base.html#module-obp.dataset.base)) and should implement three methods:
- `load_raw_data()`: Load an on-disk representation of the dataset into the module. Used during initialization.
- `pre_process()`: Perform any preprocessing needed to transform the raw data representation into a final representation.
- `obtain_batch_bandit_feedback()`: Return a dictionary containing (at least) keys: `["action","position","reward","pscore","context","n_rounds"]`

It is also helpful if the dataset module exposes a property `len_list`, which is how many items the bandit shows the user at a time. Often the answer is 1, though in the case of OBD it's 3.

In [1]:
import os
import obp
from src.data.obp_dataset import MovieLensDataset

In [2]:
dataset = MovieLensDataset(
    data_path=os.path.join(os.getcwd(), "data/"), 
    embedding_network_weights_path="model/pmf/emb_50_ratio_0.800000_bs_1000_e_258_wd_0.100000_lr_0.000100_trained_pmf.pt", 
    embedding_dim=50,
    users_num=943,
    items_num=1682,
    state_size=5
)

/media/bryan/ace784ad-85ec-4b04-91fc-978eed5d4b4d/drl-recsys/src/data/obp_dataset.py:54: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  self.load_raw_data()


----- Finished data load
----- Preprocessing dataset


/media/bryan/ace784ad-85ec-4b04-91fc-978eed5d4b4d/drl-recsys/src/data/obp_dataset.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df["item_id_history"] = pd.concat(df_list)


Finished preprocessing


In [3]:
bandit_feedback = dataset.obtain_batch_bandit_feedback()
print("feedback dict:")
for key, value in bandit_feedback.items():
    print(f"  {key}: {type(value)}")

feedback dict:
  n_rounds: <class 'int'>
  n_actions: <class 'int'>
  action: <class 'numpy.ndarray'>
  position: <class 'numpy.ndarray'>
  reward: <class 'numpy.ndarray'>
  pscore: <class 'numpy.ndarray'>
  context: <class 'numpy.ndarray'>
  action_context: <class 'numpy.ndarray'>


In [4]:
exp_rand_reward = round(bandit_feedback["reward"].mean(),4)
print(f"Expected reward for uniform random actions: {exp_rand_reward}")

Expected reward for uniform random actions: 0.2651


# Off-Policy Evaluation (OPE)

The next step is OPE which attempts to estimate the performance of online bandit algorithms using the logged bandit feedback and ReplayMethod(RM).

In [5]:
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression

import obp
from obp.ope import (
    OffPolicyEvaluation, 
    RegressionModel,
    InverseProbabilityWeighting as IPS,
    SelfNormalizedInverseProbabilityWeighting as SNIPS,
    DirectMethod as DM,
    DoublyRobust as DR,
    DoublyRobustWithShrinkage as DRos,
)

from src.model.simulator import run_bandit_simulation
from src.model.bandit import EpsilonGreedy, LinUCB, WFairLinUCB, FairLinUCB

In [6]:
n_groups = 10
fairness_weight = {k: 1.0 for k in range(1, n_groups + 1)}
with open("data/ml-100k/movies_groups.pkl", "rb") as pkl_file:
    movies_groups = pickle.load(pkl_file)

In [7]:
# import torch
# from torch.utils.data.dataset import Dataset, random_split
# class BanditDataSet(Dataset):
#     def __init__(self, action, reward, position, context, rounds):
#         self.action = action
#         self.reward = reward
#         self.position = position
#         self.context = context
#         self.rounds = rounds
        
#     def __getitem__(self, index):
#         return (
#             self.action[index], 
#             self.reward[index], 
#             self.position[index], 
#             self.context[index].reshape(1, 150)
#         )

#     def __len__(self):
#         return self.rounds

# bandit_dataset = BanditDataSet(
#     bandit_feedback["action"],
#     bandit_feedback["reward"],
#     bandit_feedback["position"],
#     bandit_feedback["context"],
#     bandit_feedback["n_rounds"]
# )

# kwargs = {"num_workers": 1, "pin_memory": True} 
# train_loader = torch.utils.data.DataLoader(
#     dataset=bandit_dataset, batch_size=1, shuffle=True, **kwargs
# )

In [8]:
epsilon_greedy = EpsilonGreedy(
    n_actions=dataset.n_actions,
    epsilon=0.1,
    n_group=n_groups,
    item_group=movies_groups,
    fairness_weight=fairness_weight
)
eg_action_dist, eg_aligned_cvr, eg_cvr, eg_propfair, eg_ufg, eg_group_count = run_bandit_simulation(
    bandit_feedback=bandit_feedback,
    policy=epsilon_greedy,
    epochs=5,
)

100%|██████████| 98266/98266 [00:06<00:00, 15547.60it/s]


In [9]:
lin_ucb = WFairLinUCB(
    dim=dataset.dim_context,
    n_actions=dataset.n_actions,
    epsilon=0.25,
    n_group=n_groups,
    item_group=movies_groups,
    fairness_weight=fairness_weight,
    batch_size=1
)
linucb_action_dist, linucb_aligned_cvr, linucb_cvr, linucb_propfair, linucb_ufg, linucb_group_count = run_bandit_simulation(
    bandit_feedback=bandit_feedback,
    policy=lin_ucb,
    epochs=5,
)

  3%|▎         | 3033/98266 [04:46<3:41:31,  7.16it/s]

In [ ]:
# estimate the policy value of the online bandit algorithms using RM
ope = OffPolicyEvaluation(
    bandit_feedback=bandit_feedback,
    ope_estimators=[
        IPS(estimator_name="IPS"), 
        SNIPS(estimator_name="SNIPS"),
        DM(estimator_name="DM"), 
        DR(estimator_name="DR"),
        DRos(estimator_name="DRos")
    ]
)

In [ ]:
# obp.ope.RegressionModel
regression_model = RegressionModel(
    n_actions=dataset.n_actions, # number of actions; |A|
    len_list=dataset.len_list, # number of items in a recommendation list; K
    base_model=LogisticRegression(C=100, max_iter=100000), 
)

In [ ]:
estimated_rewards = regression_model.fit_predict(
    context=bandit_feedback["context"],
    action=bandit_feedback["action"],
    reward=bandit_feedback["reward"],
    position=bandit_feedback["position"],
)

In [ ]:
eg_estimated_policy_value = ope.estimate_policy_values(
    action_dist=eg_action_dist, # \pi_e(a|x)
    estimated_rewards_by_reg_model=estimated_rewards, # \hat{q}
)
eg_estimated_policy_value

In [ ]:
linucb_estimated_policy_value = ope.estimate_policy_values(
    action_dist=linucb_action_dist, # \pi_e(a|x)
    estimated_rewards_by_reg_model=estimated_rewards, # \hat{q}
)
linucb_estimated_policy_value